#Aluno: Luiz Carlos Ferreira Carvalho
#DRE: 120025788

*IMPORTS/CACHE*

In [1]:
#@title
import numpy as np
from fractions import Fraction
from functools import lru_cache, wraps
from random import randint

#Código para cachear função e otimizar a performance
def cache(f):

    def g(*args):
        if args not in g.cache:
            g.cache[args] = f(*args)
        return g.cache[args]
    g.cache = {}
    g.__doc__  = f.__doc__
    g.__name__ = f.__name__
    return g

#Código para cachear função que recebe um array e otimizar a performance
def np_cache(function):
    @lru_cache()
    def cached_wrapper(hashable_array):
        array = np.array(hashable_array)
        return function(array)

    @wraps(function)
    def wrapper(array):
        return cached_wrapper(tuple(array))

    wrapper.cache_info = cached_wrapper.cache_info
    wrapper.cache_clear = cached_wrapper.cache_clear

    return wrapper


**LETRA A**

In [41]:
#@title
@cache
def linhaMatrizTransicao(linha):

    linhaMarkov = np.zeros([123])

    probsCasas = [0, 0, 1/18, 1/18, 1/9, 1/9, 1/6, 1/9, 1/9, 1/18, 1/18, 0]

    if (linha <= 119):
        linhaAux = linha % 40
        inicio = (linhaAux * 1)
        over = (12 + inicio) - 40
        for i in range(40):
            if (i > inicio) and (i <= (inicio + 12)):
                linhaMarkov[i] = probsCasas[i-linhaAux-1]
            if (over >= 0) and (i <= over):
                linhaMarkov[i] = probsCasas[len(probsCasas) - 1 - over + i]

        if (linha < 80):
            linhaMarkov[linha + 40] = 1/6

        if (linha >= 80 and linha <= 119):
            linhaMarkov[120] = 1/6

    else:
        if (linha == 120):
            linhaMarkov[20] = 1/6
            linhaMarkov[121] = 5/6

        if (linha == 121):
            linhaMarkov[20] = 1/6
            linhaMarkov[122] = 5/6

        if (linha == 122):
            linhaMarkov[20] = 1
        
    return linhaMarkov

def imprimeLinhaMatriz(estado):
    linha = linhaMatrizTransicao(estado)

    linhaAux = []
    for j in range(len(linha)):
        linhaAux.append(str(Fraction(linha[j]).limit_denominator()))

    print('Linha do estado ' + str(estado))
    print(''.join(['{:6}'.format('' + str(item) + '  ') for item in range(0, 123)]))
    print(''.join(['{:6}'.format(item) for item in linhaAux]))

imprimeLinhaMatriz(50)

Linha do estado 50
0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39    40    41    42    43    44    45    46    47    48    49    50    51    52    53    54    55    56    57    58    59    60    61    62    63    64    65    66    67    68    69    70    71    72    73    74    75    76    77    78    79    80    81    82    83    84    85    86    87    88    89    90    91    92    93    94    95    96    97    98    99    100   101   102   103   104   105   106   107   108   109   110   111   112   113   114   115   116   117   118   119   120   121   122   
0     0     0     0     0     0     0     0     0     0     0     0     0     1/18  1/18  1/9   1/9   1/6   1/9   1/9   1/18  1/18  0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0 

**LETRA B**

In [44]:
#@title
@cache
def probablidades(estado:int):

    linhaMatriz = linhaMatrizTransicao(estado)

    estadosPossiveis = dict()

    for i, p in enumerate(linhaMatriz):
        if (p != 0):
            estadosPossiveis[i] = p

    array = []

    for chave in estadosPossiveis:
        probabilidade = Fraction(estadosPossiveis.get(chave)).limit_denominator()
        k = probabilidade.numerator
        if (probabilidade.denominator != 36):
            k = (36//probabilidade.denominator) * probabilidade.numerator
        
        array.extend([chave]*k)

    return array

def proximoEstado(estado:int):
    
    probs = probablidades(estado)

    novoEstado = probs[randint(0, 35)]

    return novoEstado

def simulacao(t:int, imprimir=True, estadoInicial=0):
    jogadas = [estadoInicial]
    jogadasExibicao=[str(estadoInicial)]

    contador = 1
    while contador < t:
        jogada = proximoEstado(jogadas[contador-1])
        if (imprimir == True):
          if (jogada > 119):
              jogadasExibicao.append("20p" + ((jogada % 40) * "'") )
          else:
              jogadasExibicao.append(str(jogada%40) + ((jogada // 40) * "'") )
        jogadas.append(jogada)
        contador += 1

    if (imprimir == True):
        print(jogadasExibicao)

    return jogadas

s = simulacao(10000)

['0', "0'", '7', '15', '21', '26', '35', '4', '12', '19', '25', '34', '38', '5', '10', '17', '26', '36', '3', "3'", '12', '21', '31', "31'", '37', '2', "2'", '9', '14', '21', '31', '0', '8', '18', '23', '30', '39', '7', '16', "16'", '27', '32', '39', '7', '14', '22', '25', '30', '39', '4', '14', '21', "21'", '26', '37', '0', '7', "7'", "7''", '20p', "20p'", '20', '24', '31', "31'", '34', "34'", '0', '3', '14', '21', '30', '35', '2', '11', '19', '29', "29'", '39', '9', '18', '29', '34', '0', "0'", '7', "7'", "7''", '16', '23', "23'", '32', '3', '12', "12'", '20', "20'", "20''", '26', "26'", '29', "29'", '37', '5', '11', '21', '32', '37', '1', "1'", '7', '13', "13'", '19', '22', '32', '1', '7', '14', '19', '28', '32', '38', '1', '5', '12', '15', "15'", '20', '23', '30', '39', '10', '19', '27', '30', '35', '1', '4', "4'", '15', "15'", "15''", '20p', "20p'", '20', '25', '30', '37', "37'", '4', '8', '19', '28', '35', '0', '7', "7'", '18', '24', '32', '37', '1', '6', '13', '22', '33', '37', 

**LETRA C**

In [37]:
#@title
@np_cache
def passosAtePrisao(simulacao):

    contador = 0
    preso = False

    for j in simulacao:
        if (j == 120):
            preso = True
            break
        elif (j < 40):
            contador += 1

    return contador if preso == True else 0

def media(t, imprimir=True):
  y = []

  contador = 0
  while contador < t:
      s = simulacao(10000, False)
      p = passosAtePrisao(s)

      y.append(p)

      contador += 1

  if (imprimir):
    print(y)

  return sum(y)/len(y)

e = media(1000, False)
print("E(Y): " + str(round(e)))

E(Y): 217


**LETRA D**

**LETRA E**

In [46]:
#@title
def matrizTransicao():

    matriz = np.zeros([123, 123])
    for i in range(0, 123):
        linha = linhaMatrizTransicao(i)
        matriz[i] = linha

    return matriz

def probablidadesAposTempo(t:int):
    m = matrizTransicao()

    posicao = np.zeros([123])
    posicao[0] = 1

    contador = 0
    while contador < t:
        posicao = posicao.dot(m)
        contador += 1

    probablidades = []

    for i in range(0, 123):
        z = posicao[i]

        if (i >= 40 and i < 120):
            probablidades[i%40] += z
        elif (i >= 120):
            probablidades[20] += z
        else:
            probablidades.append(z)

    return probablidades

def probabilidadePosicaoZ(z, t=1000000):
    probablidades = probablidadesAposTempo(t)

    p = probablidades[z]

    return p

def esperancaZ(probablidades):

    e = 0
    for i in range(len(probablidades)):
        e += (i*probablidades[i])

    return round(e)

#z = probPosicaoZ(20, 100000)

probabilidades = probablidadesAposTempo(10000)

print(probabilidades)

e = esperancaZ(probabilidades)

print(e)

[0.02467988208940809, 0.024664198447949908, 0.02466028499655648, 0.024648396083852938, 0.024634861825702806, 0.024620788400229982, 0.02459653201647892, 0.02457866704770447, 0.024559082261528434, 0.024544639036735195, 0.0245292317770443, 0.024515203684677855, 0.024499879332269044, 0.024483386916049617, 0.02446658711667664, 0.024449356714021302, 0.0244329499269092, 0.024416466841842493, 0.02440074670938412, 0.02438488094409046, 0.03865625995633898, 0.024352901101980726, 0.024336643802259222, 0.024624603279588837, 0.024608334449370516, 0.0248964552243575, 0.02490057616205144, 0.025209008711133567, 0.02494931856794078, 0.02499519223708893, 0.02479870238796022, 0.024871420421462103, 0.024685747011149884, 0.024737150426299724, 0.024818775239738224, 0.024799113406921484, 0.02480877074941528, 0.024755854768010246, 0.024739868266280968, 0.024689281661522646]
20
